# Train Driving Model

In [2]:
import os
import csv

import cv2
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
samples = []
with open('../data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    # Skip header line
    header = next(reader)
    for line in reader:
        samples.append(line)

In [4]:
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [5]:
def generator(samples, batch_size=32):
    num_samples = len(samples) - (len(samples) % batch_size)
    shuffle(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = '../data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #yield X_train, y_train
            yield sklearn.utils.shuffle(X_train, y_train)

In [6]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [25]:
from keras.models import Sequential
from keras.layers import Cropping2D, Dense, Flatten, Lambda, Conv2D, Convolution2D, Dropout
from keras.layers.pooling import MaxPooling2D

In [28]:
ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(160, 320, 3)))
model.add(Convolution2D(6, 5, 5, activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

In [29]:
model.compile(loss='mse', optimizer='adam')
history = model.fit_generator(
    train_generator,
    samples_per_epoch= len(train_samples),
    validation_data=validation_generator, 
    nb_val_samples=len(validation_samples),
    nb_epoch=1,
) 

Epoch 1/1
6400/6428 [============================>.] - ETA: 1s - loss: 0.7301

/usr/local/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6432/6428 [==============================] - 328s - loss: 0.7265 - val_loss: 0.0168


In [30]:
model.save('./model.ver-1.h5')

In [31]:
history.history

{'loss': [0.72653369092155451], 'val_loss': [0.016801777962303043]}